#  ANOVA

In statistics, **Analysis of Variance (ANOVA)** is also used to analyze the differences among group means. The difference between t-test and ANOVA is the former is ued to compare two groups whereas the latter is used to compare three or more groups. [Read more about the difference between t-test and ANOVA](http://b.link/anova24).

From the ANOVA test, you receive two numbers. The first number is called the **F-value** which indicates whether your null-hypothesis can be rejected. The critical F-value that rejects the null-hypothesis varies according to the number of total subjects and the number of subject groups in your experiment. In [this table](http://b.link/eda14) you can find the critical values of the F distribution. **If you are confused by the massive F-distribution table, don't worry. Skip F-value for now and study it at a later time. In this challenge you only need to look at the p-value.**

The p-value is another number yielded by ANOVA which already takes the number of total subjects and the number of experiment groups into consideration. **Typically if your p-value is less than 0.05, you can declare the null-hypothesis is rejected.**

In this challenge, we want to understand whether there are significant differences among various types of pokemons' `Total` value, i.e. Grass vs Poison vs Fire vs Dragon... There are many types of pokemons which makes it a perfect use case for ANOVA. 

In [3]:
import pandas as pd
import numpy as np

In [5]:
pokemon_data=pd.read_csv('pokemon.txt',sep=',')
pokemon_data

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


**To achieve our goal, we use three steps:**

1. **Extract the unique values of the pokemon types.**

1. **Select dataframes for each unique pokemon type.**

1. **Conduct ANOVA analysis across the pokemon types.**

#### First let's obtain the unique values of the pokemon types. These values should be extracted from Type 1 and Type 2 aggregated. Assign the unique values to a variable called `unique_types`.

*Hint: the correct number of unique types is 19 including `NaN`. You can disregard `NaN` in next step.*

In [6]:
unique_types_1 = pokemon_data['Type 1'].unique()
unique_types_2 = pokemon_data['Type 2'].unique()

unique_types = list(set(unique_types_1) | set(unique_types_2))

unique_types = [type_ for type_ in unique_types if type(type_) == str]

unique_types


['Dragon',
 'Grass',
 'Water',
 'Ghost',
 'Ice',
 'Fighting',
 'Dark',
 'Fairy',
 'Steel',
 'Flying',
 'Ground',
 'Bug',
 'Electric',
 'Fire',
 'Poison',
 'Rock',
 'Normal',
 'Psychic']

#### Second we will create a list named `pokemon_totals` to contain the `Total` values of each unique type of pokemons.

Why we use a list instead of a dictionary to store the pokemon `Total`? It's because ANOVA only tells us whether there is a significant difference of the group means but does not tell which group(s) are significantly different. Therefore, we don't need know which `Total` belongs to which pokemon type.

*Hints:*

* Loop through `unique_types` and append the selected type's `Total` to `pokemon_groups`. Be sure to loop through BOTH `Type 1` and `Type 2` to cover all occurrances of each unique type.
* Skip the `NaN` value in `unique_types`. `NaN` is a `float` variable which you can find out by using `type()`. The valid pokemon type values are all of the `str` type.
* At the end, the length of your `pokemon_totals` should be 18.

In [9]:
pokemon_totals_concat = []

for type_ in unique_types:
    total_values_type_1 = pokemon_data[pokemon_data['Type 1'] == type_]['Total']
    total_values_type_2 = pokemon_data[pokemon_data['Type 2'] == type_]['Total']
    
    combined_totals = pd.concat([total_values_type_1, total_values_type_2], ignore_index=True)
    pokemon_totals_concat.append(combined_totals)

pokemon_totals_concat


[0     300
 1     420
 2     600
 3     490
 4     590
 5     300
 6     420
 7     600
 8     700
 9     600
 10    700
 11    600
 12    700
 13    680
 14    780
 15    300
 16    410
 17    600
 18    700
 19    320
 20    410
 21    540
 22    485
 23    680
 24    680
 25    660
 26    700
 27    700
 28    300
 29    452
 30    600
 31    600
 32    634
 33    610
 34    540
 35    630
 36    340
 37    520
 38    680
 39    680
 40    680
 41    680
 42    300
 43    420
 44    600
 45    494
 46    362
 47    521
 48    245
 49    535
 Name: Total, dtype: int64,
 0     318
 1     405
 2     525
 3     625
 4     320
      ... 
 90    335
 91    494
 92    494
 93    494
 94    494
 Name: Total, Length: 95, dtype: int64,
 0      314
 1      405
 2      530
 3      630
 4      320
       ... 
 121    520
 122    306
 123    500
 124    320
 125    600
 Name: Total, Length: 126, dtype: int64,
 0     310
 1     405
 2     500
 3     600
 4     435
 5     295
 6     455
 7     555


In [11]:
summary_df = pd.DataFrame({
    'Type': unique_types,
    'Count': [len(total) for total in pokemon_totals_concat],
    'Mean Total': [total.mean() for total in pokemon_totals_concat],
    'Standard Deviation': [total.std() for total in pokemon_totals_concat],
    'Sum Total':[total.sum() for total in pokemon_totals_concat]
})


summary_df

,Type,Count,Mean Total,Standard Deviation,Sum Total
0,Dragon,50,541.760000,143.146929,27088
1,Grass,95,417.926316,103.225661,39703
2,Water,126,429.095238,111.978671,54066
3,Ghost,46,436.869565,106.301376,20096
4,Ice,38,467.447368,116.834070,17763
5,Fighting,53,470.113208,113.987459,24916
6,Dark,51,460.901961,113.144908,23506
7,Fairy,40,415.925000,135.741532,16637
8,Steel,49,486.591837,103.703487,23843
9,Flying,101,453.831683,123.496483,45837


#### Now we run ANOVA test on `pokemon_totals`.

*Hints:*

* To conduct ANOVA, you can use `scipy.stats.f_oneway()`. Here's the [reference](http://b.link/scipy44).

* What if `f_oneway` throws an error because it does not accept `pokemon_totals` as a list? The trick is to add a `*` in front of `pokemon_totals`, e.g. `stats.f_oneway(*pokemon_groups)`. This trick breaks the list and supplies each list item as a parameter for `f_oneway`.

In [13]:
from scipy.stats import f_oneway

f_statistic, p_value_anova = f_oneway(*pokemon_totals_concat)
p_value_anova

2.6457458815984803e-15

#### Interpret the ANOVA test result. Is the difference significant?

Given a common significance level of α=0.05:

The obtained p-value is significantly less than α.

Conclusion:
There are statistically significant differences among the mean Total values of the different Pokémon types. This indicates that not all Pokémon types have the same average Total stat.